### Начинаем учебные соревнования 

13 марта 2023 года, https://www.kaggle.com/

### Открыть файлы

1) Посмотреть файлы в архиве zip (имя и размер)

In [1]:
from zipfile import ZipFile

ZipFile('/Users/tony_builder/Documents/GitHub/kaggle/datas/store-sales-time-series-forecasting.zip').infolist()

[<ZipInfo filename='holidays_events.csv' compress_type=deflate file_size=22309 compress_size=2232>,
 <ZipInfo filename='oil.csv' compress_type=deflate file_size=20580 compress_size=5822>,
 <ZipInfo filename='sample_submission.csv' compress_type=deflate file_size=342153 compress_size=65352>,
 <ZipInfo filename='stores.csv' compress_type=deflate file_size=1387 compress_size=490>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=1022269 compress_size=152503>,
 <ZipInfo filename='train.csv' compress_type=deflate file_size=121800373 compress_size=21820633>,
 <ZipInfo filename='transactions.csv' compress_type=deflate file_size=1552637 compress_size=368309>]

2. Считать конкретный файл в zip архиве и прочитать его (например через pandas.read)

In [9]:
import pandas as pd

with ZipFile('/Users/tony_builder/Documents/GitHub/kaggle/datas/store-sales-time-series-forecasting.zip', 'r') \
    as zip_file:
    holiday_df = pd.read_csv(zip_file.open('holidays_events.csv'))
    oil_df = pd.read_csv(zip_file.open('oil.csv'))
    sample_submission_df = pd.read_csv(zip_file.open('sample_submission.csv'))
    test = pd.read_csv(zip_file.open('test.csv'))
    train = pd.read_csv(zip_file.open('train.csv'))
    transactions = pd.read_csv(zip_file.open('transactions.csv'))
    stores_df = pd.read_csv(zip_file.open('stores.csv'))



### Задать цель

Описание: 
1) есть тренировочные данные, содержащие продукцию, о которых известно:
- идентификационный номер товара
- дата его продажи
- место его продажи (номер магазина)
- к чему относится этот товар (авто, красота, морепродукты, книги и пр)
- рекламировался ли он в конкретную дату
- какой объем продаж товара сделан в конкретную дату
2) Целевая переменная - непосредственно количество продаж товара
3) вспомогательные датасэты содержат дополнительную инфу (например об общей выручке магазина, которую он делал в конкретную дату) или информация о праздниках

Файл для предсказаний: информация о продаваемых товарах в последующие 15 дней

Что нужно:

        предсказать целевую переменную (объем продаж) некоторых семейств товаров в конкретные даты и в конкретных магазинах, зная информацию о том, как будет рекламироваться товар в эти даты

Информация о вспомогательных датасэтах: 

Стратегия:
1) наполнить тренировочный датасэт данными:
- информацию о том, является ли день продажи праздничным (теоретически, в праздничные дни продажи выше, чем в обычные)
- информация о стоимости на нефть в текущий день - т.к продажи товаров так же могут зависеть от цены на нефть
- Заработная плата в бюджетной сфере выплачивается каждые две недели 15 числа и в последний день месяца. Это может повлиять на продажи в супермаркетах.
- Землетрясение магнитудой 7,8 произошло в Эквадоре 16 апреля 2016 года. Люди объединились для оказания помощи, пожертвовав воду и другие предметы первой необходимости, что сильно повлияло на продажи в супермаркетах в течение нескольких недель после землетрясения.

In [148]:

holiday_df['datetime'] = pd.to_datetime(holiday_df.date, format='%Y-%m-%d')
holiday_df =  holiday_df.drop('date',axis=1)\
.rename(columns={'datetime':'date'})\
[['date', 'type', 'locale', 'locale_name', 'description', 'transferred']]

holiday_df['year'] = holiday_df.date.dt.year
holiday_df['month'] = holiday_df.date.dt.month
holiday_df['day'] = holiday_df.date.dt.day

holiday_df

,date,type,locale,locale_name,description,transferred,year,month,day
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012,4,1
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21
...,...,...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,2017,12,22
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,2017,12,23
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,2017,12,24
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,2017,12,25


Что знаем о таблице праздников:
1) Holiday 
- National: 8 перенесенных (рабочие дни для всех! ) и 52 без переноса (праздник для всех)
- Regional: 24 праздника только для некоторых ШТАТОВ
- Local: 4 перенесенных (рабочие дни для ВСЕХ!) и 133 без переноса (нерабочие только для некоторых ГОРОДОВ)

2) Event и Bridge :  61 нерабочие для всех

3) Additional: 
- National: 40 праздники для всех
- Local: 11 праздники для некоторых ГОРОДОВ

4) Transfer:
- National: 8 дней нерабочие для всех
- Local : 4 дня нерабочие только для некоторых ГОРОДОВ

5) Work day - РАБОЧИЕ для всех, даже если это суббота или воскресенье


Нужно разделить все дни на два типа: **праздничный/непраздничный**

###        
        
        Разделим таблицу (350 дней) на несколько частей:
        - нерабочие для всех
        - нерабочие для городов
        - нерабочие для штатов
        - рабочие не взирая на то, что это суббота

In [213]:
# нерабочие для всех (161 день за все годы)
holidays_for_all = holiday_df[holiday_df.locale == 'National'].\
    query("transferred == False").\
    query("type != 'Work Day' ")

holidays_for_all['workday'] = holidays_for_all.transferred.map(int)

holidays_for_all.head()

,date,type,locale,locale_name,description,transferred,year,month,day,workday
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,2012,8,10,0
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,2012,10,12,0
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False,2012,11,2,0
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False,2012,11,3,0
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False,2012,12,21,0


список праздников для всех содержит дубликаты по датам, но они все государственные и неперенесенные, поэтому их все можно смело дропнуть

In [266]:
# найти все дубликаты в столбце датасэта и вывести на экран
holidays_for_all[holidays_for_all.duplicated('date')]

,date,type,locale,locale_name,description,transferred,year,month,day,workday
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False,2012,12,24,0
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False,2012,12,31,0
157,2014-12-26,Additional,National,Ecuador,Navidad+1,False,2014,12,26,0
236,2016-05-01,Event,National,Ecuador,Terremoto Manabi+15,False,2016,5,1,0
243,2016-05-07,Event,National,Ecuador,Terremoto Manabi+21,False,2016,5,7,0
245,2016-05-08,Event,National,Ecuador,Dia de la Madre,False,2016,5,8,0


In [293]:
# нерабочие для городов (148 дней за все годы)

holidays_for_cities = holiday_df[holiday_df.locale == 'Local'].\
    query("transferred == False").\
    query("type != 'Work Day' ")

holidays_for_cities['workday'] = holidays_for_cities.transferred.map(int)


holidays_for_cities.head()


# здесь дубликаты дат нельзя удалять, потому что эти даты могут быть праздничными сразу для двух городов,
# а значит для не скольких магазинов

,date,type,locale,locale_name,description,transferred,year,month,day,workday
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2,0
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12,0
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14,0
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21,0
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,2012,5,12,0


In [188]:
# нерабочие для штатов (24 дня за все годы)

holidays_for_states = holiday_df[holiday_df.locale == 'Regional']

holidays_for_states.head()

,date,type,locale,locale_name,description,transferred,year,month,day
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012,4,1
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False,2012,6,25
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False,2012,11,6
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False,2012,11,7
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2013,4,1


In [186]:
# рабочие для всех (17 дней за все годы): либо перенесенные праздники , либо замененные на рабочие дни

workdays = holiday_df[(holiday_df.type == 'Work Day') | (holiday_df.transferred == True)]
workdays.head()

,date,type,locale,locale_name,description,transferred,year,month,day
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,2012,10,9
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False,2013,1,5
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False,2013,1,12
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,2013,10,9
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,2014,10,9


### Работаем с тренировочным датасэтом

In [136]:
train['datetime'] = pd.to_datetime(train.date, format='%Y-%m-%d')
train =  train.drop('date',axis=1)\
.rename(columns={'datetime':'date'})\
[['id', 'date', 'store_nbr', 'family', 'onpromotion', 'sales']]

#train['year'] = train.date.dt.year
#train['month'] = train.date.dt.month
#train['day'] = train.date.dt.day
train['week_day'] = train.date.dt.day_of_week

train

,id,date,store_nbr,family,onpromotion,sales,week_day
0,0,2013-01-01,1,AUTOMOTIVE,0,0.000,1
1,1,2013-01-01,1,BABY CARE,0,0.000,1
2,2,2013-01-01,1,BEAUTY,0,0.000,1
3,3,2013-01-01,1,BEVERAGES,0,0.000,1
4,4,2013-01-01,1,BOOKS,0,0.000,1
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,0,438.133,1
3000884,3000884,2017-08-15,9,PREPARED FOODS,1,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,148,2419.729,1
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,8,121.000,1


Объединим данные по продажам с данными:
- о том, где находятся магазины (город,штат)
- о том, какие из этих дней являются праздничными, причем даты Holidays yужно отфильтровать в диапазон (01.01.23 - 15.08.17), потому что train data имеет именно такой диапазон

In [273]:
# примерджим в датасэт train информацию о местонахождении магазинов
train_filled = train.merge(stores_df, on='store_nbr', 
            how='outer').drop(['type', 'cluster'], axis=1)

# Отберем только тот диапазон дней, который соответствует тренировочному датасэту, уберем дубликаты дат
holiday_temp = holidays_for_all[(holidays_for_all.date <= '2017-08-15') &\
                                        (holidays_for_all.date >='2013-01-01')]\
                                        [['date', 'workday']].\
                                        drop_duplicates('date')

# добавим столбец, описывающий - выходной это день или рабочий
train_filled['workday'] = train_filled['date'].map(holiday_temp.set_index('date')['workday'])

train_filled


,id,date,store_nbr,family,onpromotion,sales,week_day,city,state,workday
0,0,2013-01-01,1,AUTOMOTIVE,0,0.000,1,Quito,Pichincha,0.0
1,1,2013-01-01,1,BABY CARE,0,0.000,1,Quito,Pichincha,0.0
2,2,2013-01-01,1,BEAUTY,0,0.000,1,Quito,Pichincha,0.0
3,3,2013-01-01,1,BEVERAGES,0,0.000,1,Quito,Pichincha,0.0
4,4,2013-01-01,1,BOOKS,0,0.000,1,Quito,Pichincha,0.0
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,0,438.133,1,Quito,Pichincha,NaN
3000884,3000884,2017-08-15,9,PREPARED FOODS,1,154.553,1,Quito,Pichincha,NaN
3000885,3000885,2017-08-15,9,PRODUCE,148,2419.729,1,Quito,Pichincha,NaN
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,8,121.000,1,Quito,Pichincha,NaN


In [ ]:
train_filled.merge()

In [291]:
holidays_for_cities.rename(columns={'locale_name': 'city'})

,date,type,locale,city,description,transferred,year,month,day
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,2012,5,12
...,...,...,...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False,2017,11,12
339,2017-12-05,Additional,Local,Quito,Fundacion de Quito-1,False,2017,12,5
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False,2017,12,8
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False,2017,12,8


In [290]:
holidays_for_cities.head()

,date,type,locale,locale_name,description,transferred,year,month,day
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,2012,3,2
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012,4,12
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012,4,14
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012,4,21
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,2012,5,12


In [289]:
holidays_for_cities.locale_name.sort_values().unique()

array(['Ambato', 'Cayambe', 'Cuenca', 'El Carmen', 'Esmeraldas',
       'Guaranda', 'Guayaquil', 'Ibarra', 'Latacunga', 'Libertad', 'Loja',
       'Machala', 'Manta', 'Puyo', 'Quevedo', 'Quito', 'Riobamba',
       'Salinas', 'Santo Domingo'], dtype=object)

In [288]:
train_filled.city.sort_values().unique()

array(['Ambato', 'Babahoyo', 'Cayambe', 'Cuenca', 'Daule', 'El Carmen',
       'Esmeraldas', 'Guaranda', 'Guayaquil', 'Ibarra', 'Latacunga',
       'Libertad', 'Loja', 'Machala', 'Manta', 'Playas', 'Puyo',
       'Quevedo', 'Quito', 'Riobamba', 'Salinas', 'Santo Domingo'],
      dtype=object)